# Tutorial 05: ConfigBuilder System

Master the configuration API for advanced solver control and performance optimization.

## Learning Objectives

By the end of this tutorial, you will understand:
- How to build solver configurations with `ConfigBuilder`
- How to choose between different solver backends (FDM, GFDM, particles)
- How to configure coupling methods (Picard, Newton, Policy Iteration)
- How to enable acceleration (JAX, GPU)
- How to compare configurations and tune performance

## The ConfigBuilder API

The `ConfigBuilder` provides a **fluent API** for creating solver configurations. It's the recommended way to configure `solve_mfg()`.

**Why use ConfigBuilder?**
- Type-safe configuration with auto-completion
- Sensible defaults for all parameters
- Validates incompatible combinations
- Documents configuration choices clearly

**Time estimate**: 20 minutes

## Step 1: Import Dependencies

In [ ]:
import matplotlib.pyplot as plt

from mfg_pde import MFGProblem, solve_mfg
from mfg_pde.factory import ConfigBuilder
from mfg_pde.geometry import BoundaryConditions, SimpleGrid1D

## Step 2: Default Configuration (Implicit)

When you call `solve_mfg()` without a config, it uses sensible defaults:
- **Coupling**: Picard fixed-point iteration
- **HJB solver**: FDM (Finite Difference Method)
- **FP solver**: FDM
- **Convergence**: max_iterations=10, tolerance=1e-3

In [ ]:
print("=" * 70)
print("TUTORIAL 05: ConfigBuilder System")
print("=" * 70)
print()

# Create domain with boundary conditions
bc = BoundaryConditions(type="periodic")
domain = SimpleGrid1D(xmin=0.0, xmax=1.0, boundary_conditions=bc)
domain.create_grid(num_points=51)  # 51 points = 50 intervals

# Create MFG problem
problem = MFGProblem(
    geometry=domain,
    T=1.0,
    Nt=50,
    sigma=0.1,
    lam=0.5,
)

print("METHOD 1: Default configuration (implicit)")
print("-" * 70)
print()

# Without config, solve_mfg uses defaults
result_default = solve_mfg(problem, verbose=False)
print(f"  Converged: {result_default.converged}")
print(f"  Iterations: {result_default.iterations}")
print()

## Step 3: Explicit Configuration

Now let's build the same configuration explicitly to see how `ConfigBuilder` works.

In [ ]:
print("METHOD 2: Explicit configuration with ConfigBuilder")
print("-" * 70)
print()

# Build configuration step-by-step
config = (
    ConfigBuilder()
    .picard(
        max_iterations=30,  # Maximum number of iterations
        tolerance=1e-4,  # Convergence tolerance
    )
    .solver_hjb(
        "fdm",  # Finite Difference Method
        scheme="upwind",  # Upwind scheme for stability
    )
    .solver_fp(
        "fdm",  # Finite Difference Method
        scheme="lax_friedrichs",  # Lax-Friedrichs scheme
    )
    .build()  # Create the configuration object
)

print("Configuration built:")
print(f"  Coupling: {config.coupling_config.method}")
print(f"  HJB solver: {config.hjb_config.backend}")
print(f"  FP solver: {config.fp_config.backend}")
print()

result_explicit = solve_mfg(problem, config=config, verbose=False)
print(f"  Converged: {result_explicit.converged}")
print(f"  Iterations: {result_explicit.iterations}")
print()

## Step 4: Advanced Solver Selection

ConfigBuilder supports multiple solver backends. Let's explore them.

### Configuration A: GFDM (Higher-Order Accuracy)

**Generalized Finite Difference Method (GFDM)**: Higher-order spatial accuracy than standard FDM.

In [ ]:
print("=" * 70)
print("ADVANCED SOLVER CONFIGURATIONS")
print("=" * 70)
print()

print("Config A: GFDM solvers (higher-order accuracy)")
print("-" * 70)

config_gfdm = (
    ConfigBuilder()
    .picard(max_iterations=30, tolerance=1e-4)
    .solver_hjb("gfdm")  # Generalized FDM
    .solver_fp("gfdm")
    .build()
)

result_gfdm = solve_mfg(problem, config=config_gfdm, verbose=False)
print(f"  Converged: {result_gfdm.converged} in {result_gfdm.iterations} iterations")
print(f"  Final error: {result_gfdm.max_error:.6e}")
print()

### Configuration B: Hybrid (FDM HJB + Particle FP)

**Best for high-dimensional problems**: Grid for backward PDE, particles for forward simulation.

In [ ]:
print("Config B: Hybrid (FDM HJB + Particle FP)")
print("-" * 70)

config_hybrid = (
    ConfigBuilder()
    .picard(max_iterations=30, tolerance=1e-4)
    .solver_hjb("fdm")
    .solver_fp("particle", num_particles=3000, kde_bandwidth="scott")
    .build()
)

result_hybrid = solve_mfg(problem, config=config_hybrid, verbose=False)
print(f"  Converged: {result_hybrid.converged} in {result_hybrid.iterations} iterations")
print(f"  Final error: {result_hybrid.max_error:.6e}")
print()

### Configuration C: Policy Iteration (for LQ Problems)

**Policy Iteration**: Faster convergence for Linear-Quadratic problems compared to Picard.

In [ ]:
print("Config C: Policy Iteration (for LQ problems)")
print("-" * 70)

config_policy = (
    ConfigBuilder()
    .policy_iteration(
        max_iterations=10,  # Typically converges faster
        tolerance=1e-5,
    )
    .solver_fp("fdm")
    .build()
)

result_policy = solve_mfg(problem, config=config_policy, verbose=False)
print(f"  Converged: {result_policy.converged} in {result_policy.iterations} iterations")
print(f"  Final error: {result_policy.max_error:.6e}")
print()

## Step 5: Acceleration Options

ConfigBuilder supports hardware acceleration for performance-critical applications.

In [ ]:
print("=" * 70)
print("ACCELERATION OPTIONS")
print("=" * 70)
print()

print("JAX Acceleration (if available):")
print("-" * 70)

try:
    import jax  # noqa: F401

    config_jax = (
        ConfigBuilder()
        .picard(max_iterations=30, tolerance=1e-4)
        .solver_hjb("fdm")
        .solver_fp("fdm")
        .acceleration("jax")  # Enable JAX acceleration
        .build()
    )

    print("  JAX available - acceleration enabled")
    result_jax = solve_mfg(problem, config=config_jax, verbose=False)
    print(f"  Converged: {result_jax.converged} in {result_jax.iterations} iterations")
    print(f"  Time: {result_jax.execution_time:.3f}s")

except ImportError:
    print("  JAX not available - skipping JAX acceleration")

print()

## Step 6: Configuration Comparison

Let's compare all the configurations we've created.

In [ ]:
print("=" * 70)
print("CONFIGURATION COMPARISON")
print("=" * 70)
print()

configurations = {
    "Default (FDM)": result_default,
    "Explicit (FDM)": result_explicit,
    "GFDM": result_gfdm,
    "Hybrid (Particle)": result_hybrid,
    "Policy Iteration": result_policy,
}

print(f"{'Configuration':<25} {'Converged':<12} {'Iterations':<12} {'Error':<12}")
print("-" * 70)

for name, result in configurations.items():
    print(f"{name:<25} {result.converged!s:<12} {result.iterations:<12} {result.max_error:.4e}")

print()

## Step 7: Visualize Convergence Comparison

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Iterations comparison
names = list(configurations.keys())
iterations = [result.iterations for result in configurations.values()]
errors = [result.max_error for result in configurations.values()]

ax1.barh(names, iterations, color="steelblue")
ax1.set_xlabel("Iterations to Convergence")
ax1.set_title("Convergence Speed Comparison")
ax1.grid(True, alpha=0.3, axis="x")

# Final error comparison
ax2.barh(names, errors, color="coral")
ax2.set_xlabel("Final Error")
ax2.set_xscale("log")
ax2.set_title("Final Error Comparison")
ax2.grid(True, alpha=0.3, axis="x")

plt.tight_layout()
plt.show()

## Step 8: Configuration Best Practices

### General Guidelines

1. **START SIMPLE**: Use default config first, then customize
2. **MATCH PROBLEM**: Choose solvers appropriate for your problem type
3. **TUNE PARAMETERS**: Adjust tolerances and iterations as needed
4. **TEST CONVERGENCE**: Always check `result.converged` flag

### Solver Selection Guide

| Problem Type              | Recommended Config                    |
|--------------------------|---------------------------------------|
| 1D/2D, smooth domain     | FDM (default)                         |
| High-order accuracy      | GFDM                                  |
| 3D+, complex geometry    | Particle FP + FDM HJB                 |
| Linear-Quadratic         | Policy Iteration                      |
| Stiff problems           | Semi-Lagrangian HJB (advanced)        |

### Performance Tips

- **Use JAX acceleration** for repeated solves
- **Use GPU backend** for particle methods (if available)
- **Reduce grid resolution** (Nx, Nt) for prototyping
- **Increase tolerance** for faster (but less accurate) solves
- **Profile before optimizing**: Measure where time is spent

## Summary

### What You Learned

1. **ConfigBuilder API**: Fluent, type-safe configuration builder
2. **Solver backends**: FDM, GFDM, particles - each with trade-offs
3. **Coupling methods**: Picard (general), Policy Iteration (LQ-specific)
4. **Acceleration**: JAX, GPU options for performance
5. **Comparison**: How to benchmark different configurations

### Key Concepts

**Three-layer configuration**:
1. **Coupling layer**: How HJB and FP interact (Picard, Newton, Policy)
2. **HJB solver**: Backward PDE (FDM, GFDM, Semi-Lagrangian)
3. **FP solver**: Forward PDE (FDM, GFDM, Particles)

**Design pattern**:
```python
config = (
    ConfigBuilder()
    .coupling_method(...)     # How to iterate
    .solver_hjb(...)          # Backward equation
    .solver_fp(...)           # Forward equation
    .acceleration(...)        # Optional: hardware acceleration
    .build()
)
```

### Configuration is About Trade-offs

- **Speed vs Accuracy**: Higher-order methods slower but more accurate
- **Memory vs Dimension**: Grids scale exponentially, particles linearly
- **Determinism vs Scalability**: Grids deterministic, particles scale better

### Next Steps

**You've completed all 5 tutorials!**

- Explore `examples/basic/` for single-concept demos
- Explore `examples/advanced/` for complex applications
- Read `docs/` for mathematical theory and API reference
- Join community discussions on GitHub

**Happy MFG solving!**